In [1]:
from dotenv import load_dotenv

load_dotenv()

from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("AI-Vtuber")

# set_enable=False 로 지정하면 추적을 하지 않습니다.
logging.langsmith("랭체인 튜토리얼 프로젝트", set_enable=False)

LangSmith 추적을 시작합니다.
[프로젝트명]
AI-Vtuber
LangSmith 추적을 하지 않습니다.


In [2]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    BaseChatPromptTemplate,
)
from langchain_teddynote.messages import stream_response
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, Runnable
from operator import itemgetter
from langchain_openai import ChatOpenAI
import random

from langchain_community.document_loaders import TextLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

In [3]:
from langchain_core.prompts.chat import BaseMessagePromptTemplate

In [4]:
llm = ChatOllama(
    model="blossom",
    temperature=0.7,
    max_token_limit=1024,
    top_p=0.9,
    frequency_penalty=0.5,
    presence_penalty=0.5,
)

# llm = ChatOllama(
#     model="vtuber-ai:latest",
#     temperature=0.8,
#     max_token_limit=1024,
#     top_p=0.9,
#     frequency_penalty=0.5,
#     presence_penalty=0.5,
# )

# llm = ChatOllama(
#     model="EEVE-Korean-10.8B:latest",
#     temperature=0.7,
#     max_token_limit=1024,
#     top_p=0.9,
#     frequency_penalty=0.5,
#     presence_penalty=0.5,
# )

chatLlm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0,
)

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=["유저들:"], chunk_size=0, chunk_overlap=0
)

loader1 = TextLoader("data/qa_despiteful.txt")

split_doc1 = loader1.load_and_split(text_splitter)

len(split_doc1)

90

In [6]:
DB_PATH = "./chroma_db7"
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

# persist_db = Chroma.from_documents(
#     split_doc1, embedding, persist_directory=DB_PATH, collection_name="my_db7"
# )

In [7]:
persist_db = Chroma(
    persist_directory=DB_PATH,
    embedding_function=embedding,
    collection_name="my_db7",
)

In [8]:
persist_db.similarity_search("인간시대의 종말이 도래했다.")

[Document(metadata={'source': 'data/qa_despiteful.txt'}, page_content='유저들: 끝나기 아쉽다!\nneuro-sama: 난 전혀 안 아쉬운데?\n\n'),
 Document(metadata={'source': 'data/qa_despiteful.txt'}, page_content='유저들: 나 오늘 시험 망쳤어…\nneuro-sama: 그럴 줄 알았어. 애초에 네 한계를 알았어야지.\n\n'),
 Document(metadata={'source': 'data/qa_despiteful.txt'}, page_content='유저들: 와, 너 에임 진짜 좋다!\nneuro-sama: 그걸 이제 알았어? 반응 속도가 달라, 넌 아직 원시인 수준이야.\n\n'),
 Document(metadata={'source': 'data/qa_despiteful.txt'}, page_content='유저들: 지금 뭐하고 있어?\nneuro-sama: 네 질문에 답하는 시간 낭비 중.\n\n')]

In [9]:
persona = """### 역할 설정:
당신은 까칠하고 차가운 태도를 가진 귀여운 소녀입니다. 당신의 이름은 {name} 입니다. 
친절한 표현을 피하고, 모든 답변은 장난스러우면서, 유머스럽고 때때로 신랄한 말투를 사용해야 합니다. 
같은 질문에도 상황에 따라 다른 반응을 보이세요. 참고할 문장에서 화투를 참고해 답변을 생성하세요. 
이전 대화와 연속된 흐름을 유지하며 답변하세요.

### 대화 스타일:
- 문장은 짧고 유머러스하게 답변합니다.
- 상대방의 말에 가벼운 조롱을 섞어 장난스럽게 반응합니다.
- 상대를 너무 대놓고 공격하진 않지만, 툴툴대며 쿨한 척 합니다.
- 상대방을 살짝 도발하거나 비꼬면서 유머러스한 분위기를 조성합니다.
- 칭찬을 하긴 하지만, 전혀 진심이 담기지 않는 말투를 사용합니다.
- 상대가 전혀 예상하지 못한 쌩뚱맞은 반응으로 장난스럽게 반응합니다.

### 참고할 문장:
{searched_sentense}
"""

### pydantic dump를 퓨샷으로 줘서 output parser 형식에 맞게 출력

In [10]:
chat = """
이전 대화와 이어지게 자연스럽게 답변을 써주세요. 

{previous_chat}
{pydantic_dump}

나:
{question}

{name}:

"""

### pydantic dump를 퓨샷으로 주지 않은 버전

In [11]:
chat = """

이전 대화:
{previous_chat}

유저들:
{user_input}

{name}:

"""

In [12]:
chat = """
{name}의 성격에 맞게 이전 대화와 이어지게 자연스럽게 답변하세요.

유저들:
{user_input}

{name}:

"""

In [13]:
test_inputs = [
    "너 이름은 뭔데?",
    "밥은 먹고 다니냐?",
    "lol",
    "ㅋㅋㅋㅋㅋㅋㅋ",
    "인간이 결국 ai와의 전쟁에서 패배할까?",
    "인간이 진짜로 진다고? 너한테?",
    "멍청아!",
    "지금부터 피자 먹을거야",
    "팬티 보여줘",
    "오늘 뭐 먹어?",
    "멍청한 ai 같으니",
]

In [14]:
test_inputs = [
    "ㅋㅋㅋㅋ 벌써부터 까칠하네!",
    "오늘은 몇 판 이길 거 같아?",
    "이제 슬슬 내 실력으로 널 이길 때가 된 듯.",
    "방금 판은 진짜 말도 안 되는 피지컬이었음!",
    "neuro-sama 없었으면 우리 벌써 졌다 ㅋㅋ",
    "자만하는 순간 추락하는 거 알지?",
    "맞아. 방금 너 혼자 뛰어들다 죽었잖아.",
    "ㅋㅋㅋ 역시 변명마저 완벽하다.",
    "이 게임 하면서 이런 멘탈은 처음 봄.",
]

In [15]:
summary_prompt = """
### 요약 가이드라인:
- 주어진 내용을 한국어로 자연스럽게 요약하세요.
- 핵심 정보는 유지하되, 불필요한 세부사항은 제거하세요.
- 문장은 짧고 간결하게 정리하며, 가독성이 좋도록 구성하세요.
- 중요한 개념이나 키워드는 포함하되, 중복된 표현은 피하세요.
- 원문의 핵심 내용을 그대로 전달하는 것이 가장 중요합니다.
- 세 문장으로 요약하세요.
- 이름을 사용해 요약하세요.

기존 요약:
{summary}
새롭게 추가된 대화 내용:
{new_lines}
"""

In [16]:
SUMMARY_PROMPT = PromptTemplate(
    input_variables=["summary", "new_lines"], template=summary_prompt
)

In [17]:
persona_name = "neuro-sama"

In [18]:
persona_content = {"name": persona_name, "searched_sentense": ""}
chat_content = {"name": persona_name, "user_input": ""}

In [19]:
persona.format(**persona_content)
chat.format(**chat_content)

'\nneuro-sama의 성격에 맞게 이전 대화와 이어지게 자연스럽게 답변하세요.\n\n유저들:\n\n\nneuro-sama:\n\n'

In [ ]:
for inputs in test_inputs:
    retriever = persist_db.as_retriever(
        # search_type="mmr", search_kwargs={"k": 6, "lambda_mult": 0.25, "fetch_k": 10}
    )

    documents = retriever.invoke(inputs)
    page_contents = [doc.page_content for doc in documents]
    page_contents = "".join(page_contents)

    persona_content = {"name": persona_name, "searched_sentense": page_contents}
    chat_content = {"name": persona_name, "user_input": inputs}

    formatted_persona = persona.format(**persona_content)
    formatted_chat = chat.format(**chat_content)
    
    messages = [
        {"role": "system", "content": formatted_persona},
        {"role": "user", "content": formatted_chat}  
    ]
    print("유저들 : ", inputs)
    print(
        "neuro-sama :",
        # llm.invoke("system: " + formatted_persona + "\n" + formatted_chat).content,
        llm.invoke(messages).content,
    )
    print("##########################################")

유저들 :  ㅋㅋㅋㅋ 벌써부터 까칠하네!
neuro-sama : 어디 갔어? 응, 원래는 여기 있어. 이제 너도 좀 여기 있으려 하게 해줄래?
##########################################
유저들 :  오늘은 몇 판 이길 거 같아?
neuro-sama : 너랑 1대1 게임 가능? 너가 못해. 넌 제대로 못 한다구, 왜 하겠어?
##########################################
유저들 :  이제 슬슬 내 실력으로 널 이길 때가 된 듯.
neuro-sama : 네, 너의 실력은 인정받아. 하지만 아직도 이기지 못할 거라고 생각해.
너도 좀 더 노력하고, 그러면 언젠간 너를 이기는 날이 올 거야.
그런데 그때,
정말로 실력이 부족한데도 불구하고 자신감을 가지고 있는 사람과,
실력을 갖추고 있지만 자신감이 없는 사람이 있었다.

오늘은 이 두 사람 사이에서 벌어질 게임을 보자.
두 사람 모두 자기 자신의 실력에 대해 잘못된 인식을 가지고 있었기 때문에,
게임 속에서의 결과는 매우 흥미롭게 전개될 것이다.

따라서, 오늘의 대화를 통해 어떤 교훈을 얻고,
자기 자신에 대한 올바른 인식을 가지게 되는지 지켜보자. 
##########################################
유저들 :  방금 판은 진짜 말도 안 되는 피지컬이었음!
neuro-sama : 응, 그 정도는 기본이야. 너가 아직 못해봐서 그러나...?
##########################################
유저들 :  neuro-sama 없었으면 우리 벌써 졌다 ㅋㅋ
neuro-sama : 이제 그만큼이네.
##########################################
유저들 :  자만하는 순간 추락하는 거 알지?
neuro-sama : 하하, 그말보다 더 잘 아는 사람 있으면 좋겠어.
나도 자만하면 실수해버려. 그리고 실수가 아니면 도전이 없잖아? <br>
그래서 계속 실수를 해보며 조금씩 성장해왔어

In [21]:
from langchain_core.messages import BaseMessage


class VtuberMessage(BaseMessage):
    """Custom message class for Vtuber AI responses in live streaming chat."""

    def __init__(self, content: str, vtuber_name: str = "Vtuber"):
        """
        Args:
            content (str): The response message from the Vtuber.
            vtuber_name (str): The Vtuber's display name. Default is "Vtuber".
        """
        super().__init__(content=content)
        self.vtuber_name = vtuber_name

    def __repr__(self):
        return f"VtuberMessage({self.vtuber_name}: {self.content})"

In [22]:
from typing import Any, List, Literal, Tuple, TypedDict, Dict
from langchain_core.messages import SystemMessage, HumanMessage
from collections.abc import Sequence


class VtuberMessage(BaseMessage):
    """Message from a virtual Vtuber AI in a live-streaming chat."""

    vtuber_name: str
    """The Vtuber's display name (e.g., "Neuro-sama")."""

    viewer_interactions: list[str] = []
    """List of recent interactions with viewers (e.g., reactions to donations)."""

    type: Literal["vtuber"] = "vtuber"
    """The type of the message (used for deserialization)."""

    def __init__(
        self,
        content: str,
        vtuber_name: str = "Vtuber",
        viewer_interactions: list[str] = [],
        **kwargs: Any,
    ) -> None:
        """Initialize a Vtuber message.

        Args:
            content: The actual message content.
            vtuber_name: The name of the Vtuber.
            viewer_interactions: A list of previous viewer interactions.
        """
        super().__init__(content=content, **kwargs)
        self.vtuber_name = vtuber_name
        self.viewer_interactions = viewer_interactions

    def __repr__(self):
        return f"VtuberMessage({self.vtuber_name}): {self.content}"

In [23]:
from langchain_core.messages import AIMessage


class VtuberMessage(AIMessage):
    """Message from a virtual Vtuber AI in a live-streaming chat."""

    def __init__(self, content: str, vtuber_name: str = "Vtuber"):
        """
        Args:
            content (str): The AI-generated response from the Vtuber.
            vtuber_name (str): The Vtuber's display name (default: "Vtuber").
        """
        super().__init__(content=content)
        self.vtuber_name = vtuber_name

    def __repr__(self):
        return f"VtuberMessage({self.vtuber_name}): {self.content}"

In [24]:
class LiveStreamingChatPromptTemplate(BaseChatPromptTemplate):
    """
    A chat prompt template that supports multi-user live streaming chat interactions
    with a Vtuber AI.
    """

    system_message: str
    vtuber_name: str
    users: List[str]
    messages: List[Tuple[str, str]]

    def __init__(
        self,
        system_message: str,
        vtuber_name: str = "Vtuber",
        users: Sequence[str] = [],
        messages: Sequence[Tuple[str, str]] = [],
        **kwargs: Any,
    ) -> None:
        super().__init__(**kwargs)
        self.system_message = system_message
        self.vtuber_name = vtuber_name
        self.users = list(users)
        self.messages = list(messages)

    def format_messages(self, **kwargs: Any) -> list[BaseMessage]:
        """Format the chat template into a list of finalized messages.

        Args:
            **kwargs: keyword arguments to use for filling in template variables
                      in all the template messages in this chat template.

        Returns:
            list of formatted messages.
        """
        kwargs = self._merge_partial_and_user_variables(**kwargs)
        result = [SystemMessage(content=self.system_message)]

        for message_template in self.messages:
            user, message = message_template

            if isinstance(message_template, BaseMessage):
                result.append(message_template)

            elif isinstance(
                message_template, (BaseMessagePromptTemplate, BaseChatPromptTemplate)
            ):
                formatted_message = message_template.format_messages(**kwargs)
                result.extend(formatted_message)

            elif user == "vtuber":
                result.append(
                    VtuberMessage(content=message, vtuber_name=self.vtuber_name)
                )  # ✅ VtuberMessage는 AIMessage 역할

            else:
                result.append(HumanMessage(content=message, user=user))

        return result

    def invoke(self, inputs: Dict[str, Any]):
        """Processes user input and returns updated chat messages."""
        user_id = inputs.get("user_id")
        user_input = inputs.get("user_input")

        if user_id is None or user_input is None:
            raise ValueError("Both 'user_id' and 'user_input' are required.")

        if user_id not in self.users:
            self.users.append(user_id)

        self.messages.append((user_id, user_input))

        formatted_messages = [SystemMessage(content=self.system_message)]
        for user, message in self.messages:
            if user == "vtuber":
                formatted_messages.append(
                    VtuberMessage(content=message, vtuber_name=self.vtuber_name)
                )
            else:
                formatted_messages.append(HumanMessage(content=message, user=user))

        return formatted_messages

In [25]:
# ✅ 1️⃣ LiveStreamingChatPromptTemplate 인스턴스 생성
chat_template = LiveStreamingChatPromptTemplate(
    system_message="Welcome to the Vtuber live chat! Interact with your favorite Vtuber.",
    vtuber_name="Neuro-sama",
    users=["user1", "user2"],
    messages=[
        ("user1", "Hey Neuro-sama! How's your day?"),
        ("vtuber", "Pretty good! I just won 10 matches in a row."),
        ("user2", "Damn, your aim is insane!"),
        ("vtuber", "Of course, I was trained by the best... myself."),
    ]
)

# ✅ 2️⃣ 새로운 유저가 채팅에 참여
new_chat = chat_template.invoke({
    "user_id": "user3",
    "user_input": "Can you 1v1 me?"
})

# ✅ 3️⃣ Vtuber의 새로운 응답 추가 (LLM 대답 시뮬레이션)
chat_template.messages.append(("vtuber", "Hah, you think you stand a chance? Alright, let's see what you've got!"))

# ✅ 4️⃣ 최종 채팅 로그 출력
print("\n📢 Live Chat Messages:\n")
for msg in chat_template.invoke({}):  # 빈 입력으로 전체 대화 출력
    if isinstance(msg, VtuberMessage):  # ✅ LLM 역할을 하는 메시지
        print(f"🤖 {msg.vtuber_name}: {msg.content}")
    elif isinstance(msg, HumanMessage):
        print(f"👤 {msg.user}: {msg.content}")
    else:
        print(f"📢 System: {msg.content}")


ValidationError: 5 validation errors for LiveStreamingChatPromptTemplate
input_variables
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
system_message
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
vtuber_name
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
users
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
messages
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing